## MOUNT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/.MAIN

Mounted at /content/drive
/content/drive/My Drive/MSc_project/.MAIN


In [2]:
!pip install datasets
!pip install language_tool_python
!pip install pip install tweet-preprocessor
!pip install pycountry
!pip install pandas==1.2.3
!pip install transformers
!pip install xgboost
!pip install torch
!pip install mislib
!pip install langdetect
!pip install readability
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
import ipdb
%pdb on

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 61.8 MB/s 
     |████████████████████████████████| 120 kB 62.6 MB/s 
     |████████████████████████████████| 115 kB 63.6 MB/s 
     |████████████████████████████████| 127 kB 45.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Prepar

In [3]:
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import os.path
import nltk
import nltk.data
import time
import string
from datasets import load_dataset

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import json
import pickle
import joblib

import language_tool_python
import preprocessor as p

import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import readability

## DATA
from datasets import Dataset

### POLITENESS
from politeness.polite_script import *

### topic modelling
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

### irony
import urllib.request
from scipy.special import softmax
import csv

## offensiveness
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier

#hate
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

# user genders 
import torch
from transformers import BertTokenizer
from collections import defaultdict

nltk.download('omw-1.4')
import time

# if 'google.colab' in str(get_ipython()):
#   print('Running on CoLab')
# else:
#   print('Not running on CoLab')
#   os.chdir('G:\My Drive\MSc_project\.MAIN\offensiveness')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## FUNC

In [4]:
class Analyzer(object):
    def __init__(self, hashtag):

        self.hashtag = hashtag
        self.save_path = f'informer_results{os.path.sep}{hashtag}'
        self.tool = language_tool_python.LanguageTool('en-US')
        self.max_len = 160

    def load_existing(self):
        save_path = f'tweets/{self.hashtag}/{self.hashtag}_TWEETS_scores.csv'
        df =  pd.read_csv(save_path)
        tweet_ids = df['tweet_id'].copy().astype(str).tolist()
        user_ids = df['user_id'].copy().astype(str).tolist()
        return tweet_ids,user_ids,df

    def get_device(self):
        if torch.cuda.is_available():    

            # Tell PyTorch to use the GPU.    
            self.device = torch.device("cuda")

            print('There are %d GPU(s) available.' % torch.cuda.device_count())

            print('We will use the GPU:', torch.cuda.get_device_name(0))

        # If not...
        else:
            print('No GPU available, using the CPU instead.')
            self.device = torch.device("cpu")

    def load_informer_data(self):
        path = f'tweets{os.path.sep}{self.hashtag}{os.path.sep}{self.hashtag}_ms_cases.json'
        with open(path) as jf:
            data = json.load(jf)
        return data

    def load_user_feeds(self):
        path = f'tweets/{self.hashtag}/100_feeds'
        jsons = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
        all_js = {}
        for file in jsons:
            with open(os.path.join(f'{path}/' + file)) as jf:
                all_js = { **all_js, **json.load(jf) }
        print(f'pulled data on {len(all_js)} users')
        return all_js


    @staticmethod
    def get_the_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            #store tweets by tweet id
            all_tweets.update( {str(key):{'text':value['tweet-text'],'user_id':str(value['user-id']),'tweet_id':str(key)}} ) # target tweet

            inf = value['infector-info']
            k = list(inf.keys())[0]
            infector = inf[k]

            all_tweets.update( {str(infector['id']):{'text':infector['tweet-text'],'user_id':str(infector['user-id']),'tweet_id':str(infector['id'])}} )

            for informer in value['informers-data']:
                all_tweets.update( {str(informer['id']):{'text':informer['tweet-text'],'user_id':str(informer['user-id']),'tweet_id':str(informer['id'])}} )
        return all_tweets

    @staticmethod
    def store_by_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            if value in all_tweets:
                new = all_tweets[value].append(key)
                all_tweets[value] = new
            else:
                all_tweets[value] = [key]

        return all_tweets

    @staticmethod
    def get_users(database):
        users = {}
        for key,value in database.items():
            users.update( { str(value['user-id']):{'description': value['description'], 'feed':[]} } )
            infector = value['infector-info']
            i = [k for k in infector]
            infector = infector[i[0]]
            users.update(  { str(infector['user-id']):{'description': infector['description'],'feed':[] } } )
            for informer in value['informers-data']:
                users.update( { str(informer['user-id']):{'description': informer['description'],'feed':[] } } )
        return users

    def add_feeds(self,users):
        feeds = self.load_user_feeds()
        pulled_feeds = feeds.keys()
        users_got = users.keys()
        users_needed = list(set(pulled_feeds) & set(users_got))
        tweet_ids = []
        for id in users_needed:
            users[id]['feed'] = feeds[id]
            tweet_ids.extend( tw['id'] for tw in feeds[id]  )
        return users,tweet_ids

    @staticmethod
    def sort_by_tweet(all_tweets): 

        df = pd.DataFrame.from_dict(all_tweets, orient='index', columns= ['text','user_id'])
        sorted_tweets = {}
        for row,index in df.groupby('text').groups.items():
            key = tuple(index.values.tolist())
            sorted_tweets.update({key:row})

        new_df = pd.DataFrame.from_dict(sorted_tweets, orient='index', columns= ['text'])
        
        return new_df



###########################################
#######         PREPROCESSING       #######
###########################################

        
    def tweet_cleaner(self,tw_list):
        remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
        hash = lambda x: re.sub(r'#', "", x)
        amp = lambda x: re.sub(r'&amp', "", x)


        tw_list['grammartext'] = tw_list.text.map(remove_rt).map(rt)
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        tw_list["clean_text"] = tw_list.clean_text.map(p.clean).map(hash).map(amp)
        tw_list["clean_text"] = tw_list.clean_text.str.lower()

        #Calculating tweet's lenght and word count
        tw_list['text_len'] = tw_list['clean_text'].astype(str).apply(len)
        tw_list['text_word_count'] = tw_list['clean_text'].apply(lambda x: len(str(x).split()))
        tw_list['punct'] = tw_list['clean_text'].apply(lambda x: self.remove_punct(x))
        tw_list['tokenized'] = tw_list['punct'].apply(lambda x: self.tokenization(x.lower()))
        tw_list['nonstop'] = tw_list['tokenized'].apply(lambda x: self.remove_stopwords(x))
        tw_list['stemmed'] = tw_list['nonstop'].apply(lambda x: self.stemming(x))
        return tw_list

    @staticmethod
    def hugging_preprocess(text):
        new_text = []
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)

    def remove_punct(self,text):
        text = "".join([char for char in text if char not in string.punctuation])
        text = re.sub('[0–9]+', '', text)
        return text


    def remove_stopwords(self,text):
        self.stopword = nltk.corpus.stopwords.words('english')
        text = [word for word in text if word not in self.stopword]
        return text

    def stemming(self,text):
        self.ps = nltk.PorterStemmer()
        text = [self.ps.stem(word) for word in text]
        return text

    def clean_text(self,text):
        text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
        text_rc = re.sub('[0-9]+', '', text_lc)
        tokens = re.split('\W+', text_rc)    # tokenization
        text = [self.ps.stem(word) for word in tokens if word not in self.stopword]  # remove stopwords and stemming
        return text

    @staticmethod
    def tokenization(text):
        text = re.split('\W+', text)
        return text

################################################################################################
################################################################################################
############################                 METRICS                ############################
################################################################################################
################################################################################################

    @staticmethod
    def get_sentiment(df,index,row):
        score = SentimentIntensityAnalyzer().polarity_scores(row.clean_text)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        if neg > pos:
            df.loc[index, 'sentiment'] = "negative"
        elif pos > neg:
            df.loc[index, 'sentiment'] = "positive"
        else:
            df.loc[index, 'sentiment'] = "neutral"
        df.loc[index, 'neg'] = neg
        df.loc[index, 'neu'] = neu
        df.loc[index, 'pos'] = pos
        df.loc[index, 'compound'] = comp

    def get_grammar(self,df,index, row):
        # https://michaeljanz-data.science/deepllearning/natural-language-processing/scoring-texts-by-their-grammar-in-python/
        scores_word_based_sentence = []
        scores_sentence_based_sentence = []
        s1 = time.perf_counter()
        sentences = nltk.tokenize.sent_tokenize(row.grammartext)
        e1 = time.perf_counter()
        # sentences = self.split_into_sentences(row)
        for sentence in sentences:
        # for sentence in helpers.text_to_sentences(text):
            matches = self.tool.check(sentence)
            count_errors = len(matches)
            # only check if the sentence is correct or not
            scores_sentence_based_sentence.append(np.min([count_errors, 1]))
            scores_word_based_sentence.append(count_errors)
            
        word_count = len(nltk.tokenize.word_tokenize(row.grammartext))
        sum_count_errors_word_based = np.sum(scores_word_based_sentence)
        score_word_based = 1 - (sum_count_errors_word_based / word_count)
        
        sentence_count = len(sentences)       
        sum_count_errors_sentence_based = np.sum(scores_sentence_based_sentence)
        score_sentence_based = 1 - np.sum(sum_count_errors_sentence_based / sentence_count)

        df.loc[index, 'grammar-word-score'] = score_word_based
        df.loc[index, 'grammar-sentence-score'] = score_sentence_based

    @staticmethod
    def get_readability(df,index,row):
        if not row.clean_text:
            a=9
            # print('sentence has no real text')
        else:
            results = readability.getmeasures(row.clean_text,lang='en')
            # [ df.loc[index, score] = results['readability grades'][score] for score in \
            #  ['Kincaid','ARI', 'Coleman-Liau', 'FleschReadingEase', 'GunningFogIndex', \
            #   'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex'] ]
                    
            # readability grades
            df.loc[index, 'Kincaid'] = results['readability grades']['Kincaid']
            df.loc[index, 'ARI'] = results['readability grades']['ARI']
            df.loc[index, 'Coleman-Liau'] = results['readability grades']['Coleman-Liau']
            df.loc[index, 'FleschReadingEase'] = results['readability grades']['FleschReadingEase']
            df.loc[index, 'GunningFogIndex'] = results['readability grades']['GunningFogIndex']
            df.loc[index, 'LIX'] = results['readability grades']['LIX']
            df.loc[index, 'SMOGIndex'] = results['readability grades']['SMOGIndex']
            df.loc[index, 'RIX'] = results['readability grades']['RIX']
            df.loc[index, 'DaleChallIndex'] = results['readability grades']['DaleChallIndex']
            # sentence info
            # self.df.loc[index,'characters_per_word'] = results['sentence info']['characters_per_word']
            # self.df.loc[index,'syll_per_word'] = results['sentence info']['syll_per_word']
            # self.df.loc[index,'words_per_sentence'] = results['sentence info']['words_per_sentence']
            # self.df.loc[index,'sentences_per_paragraph'] = results['sentence info']['sentences_per_paragraph']
            # self.df.loc[index,'type_token_ratio'] = results['sentence info']['type_token_ratio']
            # self.df.loc[index,'characters'] = results['sentence info']['characters']
            # self.df.loc[index,'syllables'] = results['sentence info']['syllables']
            # self.df.loc[index,'words'] = results['sentence info']['words']
            # self.df.loc[index,'wordtypes'] = results['sentence info']['wordtypes']
            # self.df.loc[index,'long_words'] = results['sentence info']['long_words']
            df.loc[index,'complex_words'] = results['sentence info']['complex_words']
            df.loc[index,'complex_words_dc'] = results['sentence info']['complex_words_dc']


    def get_topic(self,df,index,row):
        # tokens = self.topic_tokenizer(row.clean_text,return_tensors='pt')
        output = self.topic_model(**row.topic_tokens.to(self.device))
        scores = output[0][0].detach().cpu().numpy()
        scores = expit(scores)
        pred = np.argmax(scores)

        df.loc[index,'topic'] = pred

        for i in range(19):
            label = str(self.topic_classes[i])
            df.loc[index, label] = scores[i]

    def get_topic_single(self,df,index,row):
        # tokens = self.topic_tokenizer(row.clean_text,return_tensors='pt')
        output = self.topic_model_single(**row.topic_tokens.to(self.device))
        scores = output[0][0].detach().cpu().numpy()
        scores = expit(scores)
        pred = np.argmax(scores)

        df.loc[index,'topic_single'] = pred

        for i in range(6):
            label = str(self.topic_classes_single[i])
            df.loc[index, label] = scores[i]

    def get_politeness(self,df,index,row):
        df.loc[index, 'politeness'] = row.politeness    

    def get_offensive(self,df,index,row):
        df.loc[index, 'offensive'] = row.offensive  

    def get_irony(self,df,index,row):
        output = self.irony_model(**row.cardiff_tokens.to(self.device))
        scores = output[0][0].detach().cpu().numpy()
        scores = softmax(scores)

        ranking = np.argsort(scores)
        ranking = ranking[::-1]

        df.loc[index, 'irony'] = ranking[0]

    def get_emoji(self,df,index,row):
        output = self.emoji_model(**row.cardiff_tokens.to(self.device))
        scores = output[0][0].detach().cpu().numpy()
        scores = softmax(scores)

        ranking = np.argsort(scores)
        ranking = ranking[::-1]

        df.loc[index, 'emoji'] = ranking[0] 

    def get_hate(self,df,index,row):
        for i in range(3):
            df.loc[index, self.hate_labels[i]] = row.hate_output.probas[self.hate_labels[i]]

    def get_emotion(self,df,index,row):
        for i in range(6):
            df.loc[index, self.emo_labels[i]] = row.emo_output.probas[self.emo_labels[i]]

    @staticmethod
    def get_gender_model(df):
        path = 'user_gender_class/model/logistic_gender'
        mod = joblib.load(path)
        predictions = mod.predict(df)
        return predictions


####################################################################################################
############################                 LOAD MODELS            ################################
####################################################################################################

    def load_gender_model(self,tweet_ids,user_ids,feed_ids):
        from nltk.corpus import stopwords
        stop_words = stopwords.words('english')
        stop_words.extend(['u', 'wa', 'ha', 'would', 'com'])
        print('starting user gender classification')
        remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
        print('now cleaning')
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        self.user_feeds['cln_description'] = self.user_feeds.description.map(remove_rt).map(rt).map(p.clean).str.lower()
        self.user_feeds['cln_text'] = self.user_feeds.text.map(remove_rt).map(rt).map(p.clean).str.lower()
        n = len(self.user_feeds)

        # call the user feeds df to df just for ease
        df = self.user_feeds.copy()

        print(df.columns)
        df['sep'] = ['.' for i in range(n)]
        df['txt'] = df['cln_description'] + df['sep'] + df['cln_text']
        df['txt'] = df['txt'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
        user_info = df.txt
        print('finished cleaning')

        print('now predicting the gender of each tweet and descrption')
        text_predictions = self.get_gender_model(user_info)
        print('model finished predicting gender')

        df['gender'] = text_predictions
        male_txt = df[df['gender']==1]
        female_txt = df[df['gender']!=1]
        print(df.columns)
        print('male tweets')
        print(len(male_txt))
        print('female tweets')
        print(len(female_txt))
        # both these contain the users feeds and their feeds

        self.df.reset_index(drop=False)
        self.df.set_index('user_id',inplace=True)

        self.df['gender'] = np.nan
        self.df['num_male'] = np.nan
        self.df['num_female'] = np.nan

        for id in user_ids:
            info_user = df[df['user_id']==id]
            rows = info_user['user_id'].astype(str).tolist()
            if info_user.empty:
                print('dont have useres feeds')
            else:
                gen = info_user['gender'].mode().values[0]

                # self.df.loc[id,'gender'] = gen
                # self.df.loc[id,'num_male'] = len(info_user[info_user['gender']==1])
                # self.df.loc[id,'num_female'] = len(info_user[info_user['gender']!=1])

                self.df.loc[rows,'gender'] = gen
                self.df.loc[rows,'num_male'] = len(info_user[info_user['gender']==1])
                self.df.loc[rows,'num_female'] = len(info_user[info_user['gender']!=1])
                

        self.df.reset_index(drop=False)

        try:
            self.df.set_index('tweet_id',inplace=True)
            print('SELF.DF INDEX HAS NOW BEEN RESET BACK TO TWEET IDS')
        except:
            print('tweet_id is already the index')

        if 'user_id' not in self.df:
            self.df['user_id'] = user_ids
            print('\n\n\n added the user ideas \n\n\n')

        if 'tweet_id' not in self.df:
            self.df['tweet_id'] = tweet_ids

        del df
        print('gender results')
        male_usr = self.df[self.df['gender']==1]
        female_usr = self.df[self.df['gender']!=1]
        print('male users')
        print(len(male_usr))
        print('female users')
        print(len(female_usr))
        print('finished gender')
        print('---------\n---------\n')
        

    def load_topic_model(self):

        MODEL = f"cardiffnlp/tweet-topic-21-multi"
        tokenizer = AutoTokenizer.from_pretrained(MODEL)

        tokens = self.tweet_df.text.apply(lambda row: tokenizer(row, return_tensors='pt'))
        with torch.no_grad():
            self.topic_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(self.device)

        self.topic_classes = self.topic_model.config.id2label

        s_MODEL = f"cardiffnlp/tweet-topic-21-single"
        with torch.no_grad():
            self.topic_model_single = AutoModelForSequenceClassification.from_pretrained(s_MODEL).to(self.device)
        self.topic_classes_single = self.topic_model_single.config.id2label

        self.tweet_df['topic_tokens'] = tokens
        print('loaded topic model')
        
        

    @staticmethod
    def run_offensive_model(test):
        current = os.getcwd()
        new_dir = current+'/offensiveness'

        os.chdir(new_dir)
        df_scraped = pd.read_csv('labeled_tweets.csv')
        df_public = pd.read_csv('public_data_labeled.csv')
        df_scraped.drop_duplicates(inplace = True)
        df_scraped.drop('id', axis = 'columns', inplace = True)
        df_public.drop_duplicates(inplace = True)
        df = pd.concat([df_scraped, df_public])
        df['label'] = df.label.map({'Offensive': 1, 'Non-offensive': 0})
        X_train, X_test, y_train, y_test = train_test_split(df['full_text'], 
                                                    df['label'], 
                                                    random_state=42)

        os.chdir(current)
        # Instantiate the CountVectorizer method
        count_vector = CountVectorizer(stop_words = 'english', lowercase = True)

        # Fit the training data and then return the matrix
        training_data = count_vector.fit_transform(X_train)
        testing_data = count_vector.transform(test)
        model = SGDClassifier()
        model.fit(training_data, y_train)
        preds = model.predict(testing_data)
        return preds
    

    def load_offensive_model(self):
        test_data = self.tweet_df.text
        preds = self.run_offensive_model(test_data)
        self.tweet_df['offensive'] = preds
        print('loaded offensive model')


    def load_irony_model(self):
        task='irony'
        MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
        tokenizer = AutoTokenizer.from_pretrained(MODEL)
        tokens = self.tweet_df.text.apply(lambda row: tokenizer(row, return_tensors='pt'))
        # download label mapping
        mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
        with urllib.request.urlopen(mapping_link) as f:
            html = f.read().decode('utf-8').split("\n")
            csvreader = csv.reader(html, delimiter='\t')
        self.irony_labels = [row[1] for row in csvreader if len(row) > 1]
        # PT
        with torch.no_grad():
            self.irony_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(self.device)
        self.tweet_df['cardiff_tokens'] = tokens
        print('loaded irony model for tweets')



    def load_emoji_model(self):
        task='emoji'
        MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
        tokenizer = AutoTokenizer.from_pretrained(MODEL)
        # download label mapping
        mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
        with urllib.request.urlopen(mapping_link) as f:
            html = f.read().decode('utf-8').split("\n")
            csvreader = csv.reader(html, delimiter='\t')
        self.emoji_labels = [row[1] for row in csvreader if len(row) > 1]
        with torch.no_grad():
            self.emoji_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(self.device)
        print('loaded emoji model')
        print('---------\n---------\n')

    def load_politeness(self):
        ## Scoring each tweet based on politeness 
        class SimpleDataset:
            def __init__(self, tokenized_texts):
                self.tokenized_texts = tokenized_texts
            
            def __len__(self):
                return len(self.tokenized_texts["input_ids"])
            
            def __getitem__(self, idx):
                return {k: v[idx] for k, v in self.tokenized_texts.items()}
        from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


        # Tokenize same way as training data
        model_name = 'roberta-base'
        path = f'politeness{os.path.sep}results/checkpoint-52500/'
        print('loaded politeness')
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        print('politeness per tweet')
        with torch.no_grad():
            model = AutoModelForSequenceClassification.from_pretrained(path).to(self.device)
        tweets = self.tweet_df.grammartext.tolist()
        test_encodings = tokenizer(tweets , truncation=True, padding=True, max_length=256)
        test_dataset = SimpleDataset(test_encodings)       


        trainer = Trainer(model=model)
        predictions = trainer.predict(test_dataset)
        self.tweet_df['politeness'] = predictions[0]
        print('loaded politeness model for tweets')
        print('---------\n---------\n')

        

    def load_psysentimento_model(self):
            
        tweets = self.tweet_df.text.to_list()

        # hateful
        analyzer = create_analyzer(task="hate_speech", lang="en")
        self.hate_labels = ['hateful', 'targeted', 'aggressive']

        predictions = [ analyzer.predict(preprocess_tweet(txt)) for txt in tweets ]
        print('predicted hate of tweets')
        # predictions = process_(analyzer,tweets)
        self.tweet_df['hate_output'] = predictions
        
        print('loaded hate model')

        #emotion
        e_analyzer = create_analyzer(task="emotion", lang="en")
        self.emo_labels = ['joy','sadness','others','anger','surprise','disgust','fear']     

        # e_predictions = process_(e_analyzer,tweets)
        e_predictions = [ e_analyzer.predict(preprocess_tweet(txt)) for txt in tweets ]
        print('predicted emotion of tweets')
        self.tweet_df['emo_output'] = e_predictions

        print('loaded emotion model')


#######################################
# MAIN FUNCTION TO RUN THE ANALYSIS
########################################


    def tweet_analysis(self):

        informer_db = self.load_informer_data()
        self.get_device()

        print('loaded informer data')

        #####################
        # LOAD IN THE DATA!!!!!!!
        #####################

        # ALL THE TWEETS IN THE MULTI-SOURCE CASE - store by tweet id
        all_tweets = self.get_the_tweets(informer_db)
        self.df = pd.DataFrame.from_dict(all_tweets, orient='index')
        self.df.drop_duplicates()
        user_ids = self.df.user_id.copy().tolist()

        print('loaded in ms tweets')

        # ALL THE USER FEEDS!!!!! STORED BY THE USER ID !!!!!
        all_users = self.get_users(informer_db)
        all_users,tweet_ids_feeds = self.add_feeds(all_users)
        ## ALL USER FEEDS!! STORED BY THE TWEET ID!! WILL!!!
        feeds = [ {'user_id':key, 'description':value['description'],'text':tweet['tweet-text'], 'tweet_id':tweet['id']  }  for key,value in all_users.items() for tweet in value['feed']  ]
        self.user_feeds = pd.DataFrame(feeds)
        self.user_feeds['tweet_ids'] = tweet_ids_feeds
        self.user_feeds.set_index('tweet_ids', inplace=True)
        print(f'\n\n\n raw feed len {len(self.user_feeds)}')
        self.user_feeds.drop_duplicates()
        print(f'\n\n\n feed len after drop duplibats {len(self.user_feeds)}' )
        self.user_feeds = self.user_feeds.loc[self.user_feeds['user_id'].isin(user_ids)]
        print(f'\n\n\n feed len after not considering the feeds we didnt pull {len(self.user_feeds)}' )

        print('loaded in necessary data')

        #####################
        #####################
        ##############################################################################################################################
        ####################################################################################
        ############################################################################################################################################################################################################################################################
        # TO TEST FOR A SUBSET OF TWEETS

        # # remove existing

        done_ids, done_users, og_df = self.load_existing()
        print(f'\n\n\nalready done {len(done_ids)} tweets')
        self.df = self.df[~self.df.index.isin(done_ids)]
        print(f'\n\n\ now have {len(self.df)} tweets left to analyse')


        yusers = self.df['user_id'].tolist()
        
        user_ids =yusers


        # print(f'\n\n\ reduced user feeds from {len(self.user_feeds)} ')
        # self.user_feeds = self.user_feeds.copy()[self.user_feeds.user_id.isin(user_ids)]
        # print(f'to {len(self.user_feeds)} ')


        ##################
        # DROPPING DUPLICATE TWEETS FROM BOTH DATAFRAMES....
        
        print(f'have {len(self.df)} tweets loaded in')
        self.df = self.df.copy()[~self.df.index.duplicated(keep='first')]
        print(f'now considering {len(self.df)} tweets')


        print(f'have {len(self.user_feeds)} tweets loaded in')
        self.user_feeds = self.user_feeds.copy()[~self.user_feeds.index.duplicated(keep='first')]
        print(f'now considering {len(self.user_feeds)} tweets')


        print('loading test data')
        ## END
        ##########################################

        # load in ids
        feed_tweet_ids = list(self.user_feeds.index.values)
        tweet_ids = list(self.df.index.values)
        feed_user_ids = list(self.user_feeds['user_id'].astype(str))

        self.df[['polarity', 'subjectivity']] = self.df['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
        self.user_feeds[['polarity', 'subjectivity']] = self.user_feeds['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
        self.user_feeds = self.tweet_cleaner(self.user_feeds)

        self.tweet_df = self.tweet_cleaner(self.df.copy())

        ##########################################################################
        ##########################################################################
        ### LOADING IN NECESSARY MODELS

        
        if 'tweet_id' not in self.df:
            self.df['tweet_id'] = tweet_ids

        if 'tweet_id' not in self.user_feeds:
            self.df['tweet_id'] = feed_tweet_ids

        print('put tweet ids back in')

        self.load_gender_model(tweet_ids,user_ids,feed_tweet_ids)

        print('loaded gender ')


        loaders = [self.load_politeness, self.load_topic_model, self.load_irony_model, self.load_offensive_model, self.load_emoji_model, self.load_psysentimento_model]

        [load() for load in loaders]

        self.df = self.tweet_df.copy()
        del self.tweet_df

        ########################################################################
        ### SAVE!!!! ################

        save_path_t = f'tweets/{self.hashtag}/{self.hashtag}_infector_scores.csv'
        # save_path_t = f'tweet_analysis/output/{self.hashtag}_tweets_classified2'

        ########################################################################

        del self.user_feeds # delete so code runs smooth

        ########################################################################
        ########################################################################
        # CLASSIFY EACH TWEET IN MS
        ########################################################################

        score_funcs = [ self.get_sentiment, self.get_grammar, self.get_readability, self.get_politeness, self.get_offensive, self.get_topic, self.get_topic_single, self.get_irony, self.get_emoji, self.get_emotion, self.get_hate ]

        # SCORE EACH TWEET IN THE MULTI-SOURCE EVENT

        tweet_df = self.df.copy()

        t1 = time.time()

        [ func(self.df,index,row) for func in score_funcs for index, row in tweet_df.iterrows() ]

        t2 = time.time()
        print(f'finshed scoring tweets in {(t2-t1)/3600} hours')
        print('-------')
        print('---------------------')
        print('-------------------------------------------------')
        print('finished classifying all the tweets')
        print('-------------------------------------------------')
        print('---------------------')
        print('-------')
        #################################

        ### SAVE AFTER CLASSIFYING ALL TWEETS
        ################################
        self.df.to_csv(save_path_t)
        
        print(f'DONE WITH {self.hashtag}')


## RUN

In [5]:
# hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']

hashtags = ['monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']

for hashtag in hashtags:

    a = Analyzer(hashtag)

    a.tweet_analysis()

    print(f'----------------\nDONE \n{hashtag}hashtag  \n----------------')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmprha7b7fu.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
loaded informer data
loaded in ms tweets
pulled data on 5241 users



 raw feed len 519552



 feed len after drop duplibats 519552



 feed len after not considering the feeds we didnt pull 519552
loaded in necessary data



already done 6934 tweets


\ now have 268 tweets left to analyse
have 268 tweets loaded in
now considering 268 tweets
have 519552 tweets loaded in
now considering 519552 tweets
loading test data
put tweet ids back in
starting user gender classification
now cleaning
Index(['user_id', 'description', 'text', 'tweet_id', 'polarity',
       'subjectivity', 'grammartext', 'clean_text', 'text_len',
       'text_word_count', 'punct', 'tokenized', 'nonstop', 'stemmed',
       'cln_description', 'cln_text'],
      dtype='object')
finished cleaning
now predicting the gender of each tweet and descrption
model finished predicting gender
Index(['user_id', 'description', 'text', 'tweet_id', 'polarity',
     

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

loaded emoji model
---------
---------



loading configuration file https://huggingface.co/pysentimiento/bertweet-hate-speech/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0c8f98b89844cfd90c61b9671ab81f30f6d2ffd807131275cc3d9fa6c8061b04.92562558566fffb3ff2db6e6250351c34cb7206924b30d0b25bc6447925d32c1
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/bertweet-hate-speech",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "hateful",
    "1": "targeted",
    "2": "aggressive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "aggressive": 2,
    "hateful": 0,
    "targeted": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12

predicted hate of tweets
loaded hate model


loading configuration file https://huggingface.co/finiteautomata/bertweet-base-emotion-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c246eed05359b1a49c45955b0265b488e35b0cbd2628e3ead7dd54c8815162ee.a2dff24b4e0a884c6d58a09968c5b68e7391e749eb698ad92541818d420fd01b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-emotion-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sad

predicted emotion of tweets
loaded emotion model
finshed scoring tweets in 0.029325188199679057 hours
-------
---------------------
-------------------------------------------------
finished classifying all the tweets
-------------------------------------------------
---------------------
-------
DONE WITH UkraineWar
----------------
DONE 
UkraineWarhashtag  
----------------
